In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

*Downloading session chat from Github*

In [2]:
!curl https://raw.githubusercontent.com/Lawrence-Krukrubo/GADS2020_Cloud_Practice_Project/master/meeting_13th_june.txt -o meeting_13th_june.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3862  100  3862    0     0  11919      0 --:--:-- --:--:-- --:--:-- 11919
100  3862  100  3862    0     0  11919      0 --:--:-- --:--:-- --:--:-- 11919


In [3]:
os.listdir()

['.ipynb_checkpoints',
 '17may_69_students.csv',
 '4th_session_chat.txt',
 'archive.zip',
 'attendance_16thMay.csv',
 'attendance_condensed16thMay.csv',
 'bestofrt.tsv',
 'cleaning_and_splitting_movie_genres (1).ipynb',
 'ebert_reviews',
 'example-job-posting.jpg',
 'extract_attendance.ipynb',
 'extract_attendance_13thJune.ipynb',
 'features.txt',
 'getting_data.ipynb',
 'June_analysis.ipynb',
 'meeting_13th_june.txt',
 'online-job-postings.csv',
 'p1_incomplete.csv',
 'p1_ungraded.csv',
 'p1_unsubs.csv',
 'rt-html.zip',
 'rt_html',
 'session-4356-report-5_15_2022.csv',
 'session-4356-report-5_17_2022.csv',
 'session-4356-report-6_1_2022.csv',
 'session-4356-report-6_7_2022.csv',
 'sessions_data.ipynb']

In [4]:
df = pd.read_csv('17may_69_students.csv')
df.shape

(69, 3)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Email       69 non-null     object
 1   First Name  69 non-null     object
 2   Last Name   69 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


In [6]:
for col in ['First Name', 'Last Name']:
    df[col] = df[col].apply(lambda x: x.lower().replace('_', ' ').replace('-', ' '))

In [7]:
df[['First Name', 'Last Name']].head(3)

,First Name,Last Name
0,abdullah,ajao
1,abdullahi,adedayo
2,abdulmalik,ismail


In [8]:
all_names = list(df['First Name']) + list(df['Last Name'])

In [9]:
chats = 'meeting_13th_june.txt'
with open(chats, 'r') as fyle:
    lines = fyle.readlines()

*Making the read-out texts a dataframe to easily clean all rows...*

In [10]:
def make_df(texts, col='strings'):
    dd = pd.DataFrame(texts, columns=[col])
    return dd

In [11]:
dd = make_df(lines)
dd.head()

,strings
0,19:02:03 From Lawrence Krukrubo to Everyone:\n
1,\tNigeria\n
2,19:02:11 From Lawrence Krukrubo to Everyone:\n
3,\tGhana\n
4,19:02:13 From Lawrence Krukrubo to Everyone:\n


Next, we clean the cols, convert each row to a list, explode the list and return a set of unique words

In [12]:
# First, we define a regex pattern for keeping only alphabets

import re
rgex = re.compile('[^a-zA-Z]')

In [13]:
def clean_lists_explode(df, col):
    # first clean all non-alphabetic characters
    df[col] = df[col].apply(lambda x: rgex.sub(' ', x).lower().strip())
    
    # Next split each row to a list
    df[col] = df[col].str.split(' ')
    
    # Next we explode the column of lists
    exploded = df.explode(col)[col]
    
    # next we keep a set of unique values
    exploded = set(exploded)
    
    print('Column Cleaned, Listed, Exploded!')
    
    # Finally, return a list of exploded values
    return list(exploded)

In [14]:
names_df = pd.DataFrame(all_names, columns=['names'])
names_df.head(3)

,names
0,abdullah
1,abdullahi
2,abdulmalik


In [15]:
exploded1 = clean_lists_explode(dd, 'strings')

Column Cleaned, Listed, Exploded!


In [16]:
len(exploded1)

160

In [17]:
exploded2 = clean_lists_explode(names_df, 'names')

Column Cleaned, Listed, Exploded!


In [18]:
len(exploded2)

144

Let's now keep only common values in both exploded and all_names...

In [19]:
commons = list(set(exploded1).intersection(set(exploded2)))
print(f'ln commons: {len(commons)}')

ln commons: 67


In [20]:
def find_present(commons):
    ddf = pd.DataFrame([], columns=['present'])
    
    for name in commons:
        long_name = [i for i in all_names if name in i][0]
        if long_name:
            try:
                q = f'`First Name` == "{long_name}"'
                tmp = df.query(q)[['First Name', 'Last Name']].values
                assert len(tmp) > 0
            except AssertionError:
                q = f'`Last Name` == "{long_name}"'
                tmp = df.query(q)[['First Name', 'Last Name']].values
        
            name_ = ['_'.join(i) for i in tmp][0]
            ddf = ddf.append({'present':name_}, ignore_index=True)
            
    # drop Duplicates
    ddf.drop_duplicates(inplace=True)
    
    # sort names alphabetically
    ddf.sort_values(by='present', inplace=True)
    
    # reset_index
    ddf.index = range(len(ddf))
    
    # replace empty space with underscore
    ddf.present = ddf.present.apply(lambda x: x.replace(' ','_').strip())
    
    return ddf

In [21]:
present = find_present(commons)

In [22]:
print(f'total present: {len(present)}')

total present: 34


In [23]:
present

,present
0,abdullah_ajao
1,abdullahi_adedayo
2,abdulmalik_ismail
3,adedeji_kayode
4,annabella_blankson
5,courtney_gallant
6,cyril_ocloo
7,derick_biney
8,emmanuel_bamidele
9,fuad_adebisi
